In [62]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [63]:
cs = requests.get('http://pwr.nu.edu.pk/cs-faculty/')
print(cs)
ee = requests.get('http://pwr.nu.edu.pk/ee-faculty/')
print(ee)
sh = requests.get('http://pwr.nu.edu.pk/sh-faculty/')
print(sh)
department = requests.get('http://pwr.nu.edu.pk/')
print(department)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [64]:
soup0 = BeautifulSoup(cs.content, 'html.parser')
soup1 = BeautifulSoup(ee.content, 'html.parser')
soup2 = BeautifulSoup(sh.content, 'html.parser')
soup3 = BeautifulSoup(department.content, 'html.parser')

In [65]:
department_name = []
department_name.append(soup3.find('nav',class_="menuzord").findAll('li')[36].text.split('P')[0].strip())
department_name.append(soup3.find('nav',class_="menuzord").findAll('li')[39].text.split('P')[0].strip())
department_name.append(soup3.find('nav',class_="menuzord").findAll('li')[45].text.split('F')[0].strip())

print(department_name)

['Computer Science', 'Electrical Engineering', 'Science & Humanities']


In [66]:
school_name = []
school_name.append('FAST School of Computing')
school_name.append('Department of Electrical Engineering')
school_name.append('Department of Sciences and Humanities')

In [67]:
pesahwar_faculty = []
pesahwar_faculty.append(soup0.find('div',class_="faculty-list").findAll('div', class_="faculty-member"))
pesahwar_faculty.append(soup1.find('div',class_="faculty-list").findAll('div', class_="faculty-member"))
pesahwar_faculty.append(soup2.find('div',class_="faculty-list").findAll('div', class_="faculty-member"))



In [68]:
# Create the DataFrame
faculty_Peshawar_data = pd.DataFrame(columns=['ID',
                                     'Name',
                                     'Designation',
                                     'HEC Approved PHD Supervisor',
                                     'Highest Education',
                                     'Email',
                                     'School',
                                     'Department',
                                     'Extension',
                                     'ImageURL'])

In [69]:
faculty_Peshawar_data.drop(faculty_Peshawar_data.index, inplace=True)
base_url = 'http://pwr.nu.edu.pk/'
count = 0
for dep_faculty in pesahwar_faculty:
    dep_name = department_name[count]
    school = school_name[count]
    count += 1
    for professor in dep_faculty:
        print('BIO.........')
        id = professor.find('div', class_='faculty-details').find('h2').find('a',class_='text-theme-color-2').get('href').split('id=')
        int_id = int(id[1])
        print('ID: ',int_id)
        name = str(professor.find('div', class_='faculty-details').find('h2').find('a',class_="text-theme-color-2").text.strip())
        print('Name: ',name)
        designation = str(professor.find('div', class_='faculty-details').findAll('p')[0].text.split('(On Leave)')[0].strip())
        print('Designation: ',designation)
        url = professor.find('div', class_='faculty-details').find('h2').find('a',class_='text-theme-color-2').get('href')
        profile_url = base_url + url
        r = requests.get(profile_url)
        if r.status_code == 200:
            soup2 = BeautifulSoup(r.content, 'html.parser')
            HEC = soup2.findAll('div', class_='section-content')[1].find('div', class_='row').find('h5', style="color: green").text.strip()
            if HEC:
                HEC_bool = True
                print('HEC Approved',HEC_bool)
            else:
                HEC_bool = False
                print('HEC Approved',HEC_bool)
            edu = str(soup2.find('div', class_='tab-pane fade in active').find('li').text.strip())
            if edu:
                education = edu
                print('Education: ',education)
            else:
                education = 'Not Available'
                print('Education: ',education)
        else:
            HEC_bool = 'Not Available'
            print('HEC Approved: ',HEC_bool)
            education = 'Not Available'
            print('Education: ',education)

        email = str(professor.find('div', class_='faculty-details').findAll('p')[1].text.strip())
        print('Email: ',email)
        school_str = str(school)
        print('School: ', school_str)
        print('Department:',dep_name)
        ext_array = professor.find('div', class_='faculty-details').findAll('p')[2].text.strip().split('Ext.')
        if len(ext_array)>1:
            ext_int = ext_array[1].split(')')
            if ext_int[0] == ' ':
                extension = 'Not Available'
                print('Extension: ',extension)
            else:
                extension = int(ext_int[0].strip())
                print('Extension: ',extension)
        img = professor.find('img', class_='faculty-photo').get('src')
        img_str = base_url + img
        print('Img: ',img_str)
        
        faculty_member_dataframe = {'ID':int_id,
                         'Name': name,
                         'Designation': designation,
                         'HEC Approved PHD Supervisor': HEC_bool,
                         'Highest Education': education,
                         'Email': email,
                         'School': school_str,
                         'Department':dep_name,
                         'Extension': extension,
                         'ImageURL': img_str}
        new_dataframe = pd.DataFrame([faculty_member_dataframe])
        faculty_Peshawar_data = pd.concat([faculty_Peshawar_data,new_dataframe], ignore_index=True )


BIO.........
ID:  3704
Name:  Fazl-e-Basit
Designation:  Assistant Professor & HoD
HEC Approved False
Education:  M.S(Computer Science), NUST, Rawalpindi, Pakistan (2006)
Email:  fazl.basit@nu.edu.pk
School:  FAST School of Computing
Department: Computer Science
Extension:  110
Img:  http://pwr.nu.edu.pk/images/faculty/thumb3704.jpg
BIO.........
ID:  9237
Name:  Dr. Omar Usman Khan
Designation:  Professor & Director
HEC Approved True
Education:  Ph.D. (Computer And Control Engineering), Politecnico Di Torino, Italy (2013)
Email:  omar.khan@nu.edu.pk
School:  FAST School of Computing
Department: Computer Science
Extension:  103
Img:  http://pwr.nu.edu.pk/images/faculty/thumb9237.jpg
BIO.........
ID:  9315
Name:  Dr. Hafeez Ur Rehman
Designation:  Associate Professor
HEC Approved True
Education:  Ph.D. in Computer and Control Engineering, Politecnico Di Torino, Italy (2014).
Email:  hafeez.urrehman@nu.edu.pk
School:  FAST School of Computing
Department: Computer Science
Extension:  Not A

In [70]:
faculty_Peshawar_data.to_csv('pew.csv')